### Lab 7: Sistema de Recomendacion

In [65]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import json

%matplotlib inline

In [66]:
def cost(Y, X, T):
    return(((X @ T.T - Y) ** 2) * ((Y != 0) * 1)).sum()


def gradient(Y, X, T):
    R = (Y != 0) * 1
    hip_error = (X @ T.T - Y) * R

    return (
        hip_error @ T,
        hip_error.T @ X,
    )


def adam(
        Y,
        Xo,
        To,
        fun,
        jac,
        alpha=0.001,
        beta1=0.9,
        beta2=0.999,
        epsilon=0.0000001,
        max_iter=1000
):
    xm = cp.zeros(Xo.shape)
    tm = cp.zeros(To.shape)

    xv = cp.zeros(Xo.shape)
    tv = cp.zeros(To.shape)

    X, T, t = Xo, To, 0

    while t < max_iter:
        t += 1

        xg, tg = jac(Y, X, T)

        print(f'{t} \t loss={fun(Y, X, T).item():,.2f}')

        xm = beta1 * xm + (1 - beta1) * xg
        tm = beta1 * tm + (1 - beta1) * tg

        xv = beta2 * xv + (1 - beta2) * xg * xg
        tv = beta2 * tv + (1 - beta2) * tg * tg

        xmh = xm / (1 - beta1 ** t)
        tmh = tm / (1 - beta1 ** t)

        xvh = xv / (1 - beta2 ** t)
        tvh = tv / (1 - beta2 ** t)

        X -= alpha * xmh / (cp.sqrt(xvh) + epsilon)
        T -= alpha * tmh / (cp.sqrt(tvh) + epsilon)

    return X, T

### Datasets

In [67]:
ratings = pd.read_csv("ratings_small.csv")
movies = pd.read_csv("movies_metadata.csv")
links = pd.read_csv("links_small.csv")

print(ratings.shape, movies.shape, links.shape)

(100004, 4) (45466, 24) (9125, 3)


/Users/raulcastellanos/Desktop/Universidad/S7/MLModels/venvMLM/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Descripcion de los datasets

In [68]:
print("Dataset Ratings")
print("\n")
print(ratings.dtypes)
print("\n")
print("Movies Ratings")
print("\n")
print(movies.dtypes)
print("\n")
print("Links")
print("\n")
print(links.dtypes)

Dataset Ratings


userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object


Movies Ratings


adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object


Links


movieId      int64
imdbId       int64
tmdbId     

In [69]:
print(ratings.head)

<bound method NDFrame.head of         userId  movieId  rating   timestamp
0            1       31     2.5  1260759144
1            1     1029     3.0  1260759179
2            1     1061     3.0  1260759182
3            1     1129     2.0  1260759185
4            1     1172     4.0  1260759205
...        ...      ...     ...         ...
99999      671     6268     2.5  1065579370
100000     671     6269     4.0  1065149201
100001     671     6365     4.0  1070940363
100002     671     6385     2.5  1070979663
100003     671     6565     3.5  1074784724

[100004 rows x 4 columns]>


In [70]:
print(movies.head)

<bound method NDFrame.head of        adult                              belongs_to_collection    budget  \
0      False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1      False                                                NaN  65000000   
2      False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3      False                                                NaN  16000000   
4      False  {'id': 96871, 'name': 'Father of the Bride Col...         0   
...      ...                                                ...       ...   
45461  False                                                NaN         0   
45462  False                                                NaN         0   
45463  False                                                NaN         0   
45464  False                                                NaN         0   
45465  False                                                NaN         0   

                                             

In [71]:
print(links.head)

<bound method NDFrame.head of       movieId   imdbId    tmdbId
0           1   114709     862.0
1           2   113497    8844.0
2           3   113228   15602.0
3           4   114885   31357.0
4           5   113041   11862.0
...       ...      ...       ...
9120   162672  3859980  402672.0
9121   163056  4262980  315011.0
9122   163949  2531318  391698.0
9123   164977    27660  137608.0
9124   164979  3447228  410803.0

[9125 rows x 3 columns]>


### Datos en la columna de 'Genres' como JSon

para la limpieza utilicé este link como referencia
https://towardsdatascience.com/cleaning-and-extracting-json-from-pandas-dataframes-f0c15f93cb38
github
https://github.com/mpHarm88/projects/tree/master/json

In [72]:
movies['genres'] = movies['genres'].str.replace("\'","\"")
jgenres = ["genres"]

def new_json(x):
    return json.loads(x)

for x in jgenres:
    movies[x] = movies[x].apply(new_json)

    
def new_json2(x):

    ls = []

    for y in range(len(x[0])):

        for k, v in x[0][y].items():
            ls.append(str(k)+ "_" +str(v))
   
    for z in range(len(ls)):

        if ls[z] not in movies.columns:
            movies[ls[z]] = 0
            movies[ls[z]].iloc[x.name] = 1
        else:
            movies[ls[z]].iloc[x.name] = 1
    return

for x in jgenres:
    movies[[x]].apply(new_json2, axis=1)

In [73]:
print(movies.columns)
print(movies["name_Animation"])

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'id_16', 'name_Animation', 'id_35',
       'name_Comedy', 'id_10751', 'name_Family', 'id_12', 'name_Adventure',
       'id_14', 'name_Fantasy', 'id_10749', 'name_Romance', 'id_18',
       'name_Drama', 'id_28', 'name_Action', 'id_80', 'name_Crime', 'id_53',
       'name_Thriller', 'id_27', 'name_Horror', 'id_36', 'name_History',
       'id_878', 'name_Science Fiction', 'id_9648', 'name_Mystery', 'id_10752',
       'name_War', 'id_10769', 'name_Foreign', 'id_10402', 'name_Music',
       'id_99', 'name_Documentary', 'id_37', 'name_Western', 'id_10770',
       'name_TV Movie', 'name_Carousel Productions', 'id_11176'

### Generos de interes

In [74]:
# generos de interes
gcols = movies[["genres","name_Animation", "name_Comedy", "name_Science Fiction", "name_Adventure", "name_Fantasy"]]

In [75]:
"""
movies_wide = wcols.melt(id_vars="genres", var_name='Genres', value_name='value')
movies_wide1 = movies_wide[(movies_wide != 0).all(1)]
print(movies_wide, movies_wide.shape, movies.shape)

"""


'\nmovies_wide = wcols.melt(id_vars="genres", var_name=\'Genres\', value_name=\'value\')\nmovies_wide1 = movies_wide[(movies_wide != 0).all(1)]\nprint(movies_wide, movies_wide.shape, movies.shape)\n\n'

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,name_Sentai Filmworks,id_33751,name_Odyssey Media,id_17161,name_Pulser Productions,id_18012,name_Rogue State,id_18013,name_The Cartel,id_23822
